In [1]:
import os

In [ ]:
pwd

In [2]:
os.chdir("../")

In [ ]:
pwd

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SouravHalder1996/Kidney-Disease-Classification-DeepLearning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SouravHalder1996"
os.environ["MLFLOW_TRACKING_PASSWORD"]="8146336749309d3266628fdd8cf69ff92a52f2da"


In [ ]:
# MLFLOW_TRACKING_URI=https://dagshub.com/SouravHalder1996/Kidney-Disease-Classification-DeepLearning-Project.mlflow \
# MLFLOW_TRACKING_USERNAME=SouravHalder1996 \
# MLFLOW_TRACKING_PASSWORD=8146336749309d3266628fdd8cf69ff92a52f2da \
# python script.py

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from src.Kidney_Disease_Classifier.constants import *
from src.Kidney_Disease_Classifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/kidney-dataset/",
                mlflow_uri="https://dagshub.com/SouravHalder1996/Kidney-Disease-Classification-DeepLearning-Project.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )

          return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")

            else:

                mlflow.keras.log_model(self.model, "model")

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2023-12-14 13:54:04,702: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2023-12-14 13:54:04,704: INFO: common: yaml file: params.yaml loaded successfully.]
[2023-12-14 13:54:04,705: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 125s 905ms/step - loss: 24.4802 - accuracy: 0.6901
[2023-12-14 13:56:10,329: INFO: common: json file saved at: scores.json]


2023/12/14 13:56:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-12-14 13:56:13,772: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\SOURAV~1\AppData\Local\Temp\tmpn1p07r10\model\data\model\assets
[2023-12-14 13:56:14,452: INFO: builder_impl: Assets written to: C:\Users\SOURAV~1\AppData\Local\Temp\tmpn1p07r10\model\data\model\assets]


c:\Users\Sourav Halder\anaconda3\envs\kidney-disease\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2023/12/14 13:57:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
